In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import defaultdict
import matplotlib as mpl

In [ ]:
base_dir = "/eos/home-d/dmapelli/public/latino/"

plot_config  = "Full2016v6s5"
#cut     = "boos_sig_mjjincl"
cut     = "res_sig_mjjincl"
version = "v11"

# event count
#plot_config  = "Full2018v6s5"
#cut     = "evtcntall"
#version = "evtcnt"


samples_dir = os.path.join(base_dir, plot_config, cut, "samples/" + version)

In [ ]:
samples_dir

In [ ]:
# 2018
# classes = { "VBS": 0, "Wjets":1, "top": 2, "DY": 3, "VV":4, "VVV":5, "VBF-V":6}
# signal_name = "VBS"
# sample_names = ["Wjets", "VBS", "DY", "top", "VV", "VBF-V"]
# lumi = 59.7

## 2017
#classes = { "VBS": 0, "Wjets":1, "top": 2, "DY": 3, "VV":4}
#signal_name = "VBS"
#sample_names = ["Wjets", "VBS", "DY", "top", "VV"]

## 2016
classes = { "VBS": 0, "Wjets":1, "top": 2, "DY": 3, "VVV":4}
signal_name = "VBS"
sample_names = ["Wjets", "VBS", "DY", "top", "VVV"]


samples = {}

for file in os.listdir(samples_dir):
    if os.path.isdir(os.path.join(samples_dir, file)): continue
    print(file)    
    sname = file.split("_part")[0]
    s = pickle.load(open(os.path.join(samples_dir, file), "rb"))
    s.rename(columns=lambda c: c.split(cut+"_")[1] if cut in c else c, inplace=True)
    s["sample_name"]= sname
    s["class"] = classes[sname]
    if sname == signal_name:
        s["signal"] = 1
    else: 
        s["signal"] = 0
    if sname in samples:
        samples[sname] = pd.concat([samples[sname], s], ignore_index=True)
    else:
        samples[sname] = s

In [ ]:
for s, df in samples.items():
    print(f"Sample name {s:10}, events: {len(df):10}, events weighted: {(df.weight_).sum():1},    XS total (pb): {(df.weight_).sum()/(lumi*1000) :15}")

In [ ]:
sig_events = 0
bkg_events = 0
for key, df in samples.items():
    if classes[key] == 0:
        # signal
        sig_events += len(df)
    else:
        bkg_events += len(df)
print("sig", sig_events)
print("bkg", bkg_events)



## Balancing

In [ ]:
_ = plt.hist(samples["VBS"]["weight_"], bins=100)


In [ ]:
# _ = plt.hist(samples["top"].XSWeight*130, bins=100, range=(0,1))
#plt.xscale("log")

In [ ]:
# _ = plt.hist(samples["DY"].XSWeight*130, bins=100, range=(0,1))

In [ ]:
# _ = plt.hist(samples["VBS"].XSWeight*(1/samples["VBS"].XSWeight.mean()), bins=100, range=(0,2))

In [ ]:
# _= plt.hist( samples["Wjets"].XSWeight*1000, bins=100, range=(0, 2))

In [ ]:
# _= plt.hist( (background.XSWeight *(1/ background.XSWeight.mean()))*10  , bins=100, range=(0, 10))

In [ ]:
bkg_list = []
bkg_names = [ name for name in sample_names if name != signal_name ]
for bkg_name in bkg_names:
    bkg_list.append(samples[bkg_name])

In [ ]:
background = pd.concat(bkg_list, ignore_index=True)

In [ ]:
signal = samples["VBS"]

In [ ]:
ratio_neve_bkgsignal= len(background) / len(signal)
print(ratio_neve_bkgsignal)

In [ ]:
tot_ev_weighted_sig = (signal.weight_ / signal.weight_.mean()).sum()
print("TOT signal weighted events", tot_ev_weighted_sig)

In [ ]:
rescale_factor_bkg = (tot_ev_weighted_sig *ratio_neve_bkgsignal )/ background.weight_.sum()
print(rescale_factor_bkg)

In [ ]:
signal["weight_norm"] = signal.weight_ / signal.weight_.mean()
background["weight_norm"] = background.weight_ * rescale_factor_bkg

## Save signal and bkg samples

In [ ]:
import os

os.makedirs(os.path.join(samples_dir, "for_training"), exist_ok=True)
pickle.dump(background, open(os.path.join(samples_dir, "for_training/background_balanced.pkl"), "wb"))
pickle.dump(signal, open(os.path.join(samples_dir, "for_training/signal_balanced.pkl"), "wb"))

In [ ]:
plt.hist(background["weight_norm"], bins=100, range=(0, 100))
plt.yscale("log")

The total number of events will be manually balanced

### plot correlation matrix: moved to evaluation of the model